In [1]:
import torch
import torch.nn as nn
import torchvision

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
import os

from torch.utils.data import Dataset, DataLoader
# from torch.utils.data.sampler import Sampler
import torch.optim as optim
import sys
sys.path.append('../')

from dataset import LbpDataset, train_transforms, val_transforms, test_transforms, collate_fn, get_data
from visualize import visualize
# from rcnn_model import fasterrcnn_resnet201_fpn, FastRCNNPredictor
from engine import evaluate
import utils
from train_lbp import get_train_test_list

In [2]:
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone, _validate_trainable_layers
from torchvision.ops.feature_pyramid_network import LastLevelP6P7
from torchvision.models.detection.retinanet import RetinaNet
from torch.hub import load_state_dict_from_url
from torchvision.models.detection.backbone_utils import mobilenet_backbone

In [3]:
from torchvision.models.detection.retinanet import retinanet_resnet50_fpn
from torchvision.models.detection.faster_rcnn import fasterrcnn_mobilenet_v3_large_fpn
# model = torchvision.models.detection.fasterrcnn_mobilenet_v3_large_fpn(pretrained=True)
# model = retinanet_resnet50_fpn(pretrained=True)

In [4]:
def retinanet_resnet18_fpn(pretrained=False, progress=True,
                           num_classes=91, pretrained_backbone=True, trainable_backbone_layers=None, **kwargs):
    trainable_backbone_layers = _validate_trainable_layers(
        pretrained or pretrained_backbone, trainable_backbone_layers, 5, 3)

    if pretrained:
        # no need to download the backbone if pretrained is set
        pretrained_backbone = False
    # skip P2 because it generates too many anchors (according to their paper)
    backbone = resnet_fpn_backbone('resnet50', pretrained_backbone, returned_layers=[2, 3, 4],
                                   extra_blocks=LastLevelP6P7(256, 256), trainable_layers=trainable_backbone_layers)
    model = RetinaNet(backbone, num_classes, **kwargs)
#     if pretrained:
#         state_dict = load_state_dict_from_url(model_urls['retinanet_resnet50_fpn_coco'],
#                                               progress=progress)
#         model.load_state_dict(state_dict)
#         overwrite_eps(model, 0.0)
    return model

In [5]:
from torchvision.models.detection.anchor_utils import AnchorGenerator


In [6]:
# anchor_sizes = ((32, 64, 128, 256, 512, ), ) * 5
# print(anchor_sizes)
# aspect_ratios = ((0.5, 0.75, 1.0, 1.5, 2.0),) * len(anchor_sizes)
# aspect_ratios

In [7]:
def retinanet_mobilenet_fpn(pretrained=False, progress=True,
                           num_classes=91, pretrained_backbone=True, trainable_backbone_layers=None, **kwargs):
    trainable_backbone_layers = _validate_trainable_layers(
        pretrained or pretrained_backbone, trainable_backbone_layers, 5, 3)

    if pretrained:
        # no need to download the backbone if pretrained is set
        pretrained_backbone = False
    # skip P2 because it generates too many anchors (according to their paper)
#     backbone = resnet_fpn_backbone('resnet50', pretrained_backbone, returned_layers=[2, 3, 4],
#                                    extra_blocks=LastLevelP6P7(256, 256), trainable_layers=trainable_backbone_layers)
    anchor_sizes = ((32, 64, 128, 256), ) * 3
    aspect_ratios = ((0.5, 1.0, 2.0),) * len(anchor_sizes)
    rpn_anchor_generator=AnchorGenerator(anchor_sizes, aspect_ratios)
    
    backbone = mobilenet_backbone("mobilenet_v3_small", pretrained_backbone, True,
                                  trainable_layers=trainable_backbone_layers)
    
#     backbone = mobilenet_backbone("mobilenet_v3_large", pretrained_backbone, True,
#                                   trainable_layers=trainable_backbone_layers)    
    model = RetinaNet(backbone, num_classes, anchor_generator=rpn_anchor_generator, **kwargs)
#     if pretrained:
#         state_dict = load_state_dict_from_url(model_urls['retinanet_resnet50_fpn_coco'],
#                                               progress=progress)
#         model.load_state_dict(state_dict)
#         overwrite_eps(model, 0.0)
    return model

In [8]:
import easydict 
args = easydict.EasyDict({ "batch_size": 16, 
                          "epochs": 120, 
                          "data": 0, 
                          'lr':0.002,
                         'momentum':0.9,
                         'weight_decay':1e-4,
                         'start_epoch':0,
                         'gpu':0,
                          'workers':12,
                         'print_freq':1000,
                         'output_dir':'../trained_model/retinanet/'})

In [9]:
from pathlib import Path
path = Path(args.output_dir.split('checkpoint')[0])
path.mkdir(parents=True, exist_ok=True)  

In [10]:
ngpus_per_node = torch.cuda.device_count()
print(ngpus_per_node)
GPU_NUM = args.gpu # 원하는 GPU 번호 입력
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device)
print(device)

1
cuda:0


In [11]:
df = pd.read_csv('../../data/df.csv')
df.head()

,ID,file_name,task,bbox,xmin,ymin,w,h,label,occluded,des,cell_type
0,0,patch_images/2021.01.14/LBC424-20210111(1)/LBC...,[AS6] LBC424,"[1539, 199, 139, 211]",1539,199,139,211,C,0,NaN,ASC-US
1,1,patch_images/2021.01.14/LBC424-20210111(1)/LBC...,[AS6] LBC424,"[1337, 102, 256, 136]",1337,102,256,136,AS,0,NaN,ASC-US
2,2,patch_images/2021.01.14/LBC424-20210111(1)/LBC...,[AS6] LBC424,"[220, 619, 166, 169]",220,619,166,169,AS,0,NaN,ASC-US
3,3,patch_images/2021.01.14/LBC424-20210111(1)/LBC...,[AS6] LBC424,"[658, 1747, 191, 166]",658,1747,191,166,AS,0,NaN,ASC-US
4,4,patch_images/2021.01.14/LBC424-20210111(1)/LBC...,[AS6] LBC424,"[1571, 365, 136, 146]",1571,365,136,146,AS,0,NaN,ASC-US


In [12]:
df.label.unique()

array(['C', 'AS', 'ASC-US', 'LSIL', 'LS', 'HS', 'Negative', 'Carcinoma',
       'ASC-H', 'ASC-US with HPV infection', 'HSIL with HPV infection',
       'Candida', 'HSIL', 'AH', 'ASCUS-SIL', '판독불가',
       'LSIL with HPV infection', 'H', 'Benign atypia'], dtype=object)

In [13]:
from dataset_util import CLASS_MAPPER
df.label = df.label.apply(lambda x : CLASS_MAPPER[str(x)])
df.head()

,ID,file_name,task,bbox,xmin,ymin,w,h,label,occluded,des,cell_type
0,0,patch_images/2021.01.14/LBC424-20210111(1)/LBC...,[AS6] LBC424,"[1539, 199, 139, 211]",1539,199,139,211,Carcinoma,0,NaN,ASC-US
1,1,patch_images/2021.01.14/LBC424-20210111(1)/LBC...,[AS6] LBC424,"[1337, 102, 256, 136]",1337,102,256,136,ASC-US,0,NaN,ASC-US
2,2,patch_images/2021.01.14/LBC424-20210111(1)/LBC...,[AS6] LBC424,"[220, 619, 166, 169]",220,619,166,169,ASC-US,0,NaN,ASC-US
3,3,patch_images/2021.01.14/LBC424-20210111(1)/LBC...,[AS6] LBC424,"[658, 1747, 191, 166]",658,1747,191,166,ASC-US,0,NaN,ASC-US
4,4,patch_images/2021.01.14/LBC424-20210111(1)/LBC...,[AS6] LBC424,"[1571, 365, 136, 146]",1571,365,136,146,ASC-US,0,NaN,ASC-US


In [14]:
# df['label_id'] = df.label.apply(lambda x : 0. if 'Benign' in x or 'Negative' in x else 1.)
# df.label_id.value_counts()

In [15]:
df = pd.read_csv('../../data/df.csv')
df.head()
# Data loading code#
data_dir = '../../data/df.csv'
train_list, test_list = get_train_test_list(data_dir)
train_dataset = LbpDataset(train_list, default_path='/home/Dataset/scl/', transform=train_transforms)
test_dataset = LbpDataset(test_list, default_path='/home/Dataset/scl/', transform=val_transforms)  

total 7524 train 5643 test 1881
5643
1881


In [16]:
df.shape

(9735, 12)

In [17]:
train_sampler = torch.utils.data.RandomSampler(train_dataset)
test_sampler = torch.utils.data.SequentialSampler(test_dataset)

train_loader = DataLoader(
    train_dataset, batch_size=args.batch_size,
    sampler=train_sampler, num_workers=args.workers,
    collate_fn=utils.collate_fn)

test_loader = DataLoader(
    test_dataset, batch_size=args.batch_size,
    sampler=test_sampler, num_workers=args.workers,
    collate_fn=utils.collate_fn)

In [18]:
num_classes = 2
model = retinanet_mobilenet_fpn(pretrained=False, min_size=800, max_size=800, num_classes=2)
# model = retinanet_resnet50_fpn(pretrained=False, min_size=1024, max_size=1024, num_classes=2)
# model = retinanet_resnet18_fpn(pretrained=False, min_size=1024, max_size=1024, num_classes=2)

device = torch.device('cuda')
model.to(device)
print('model is loaded to gpu')

model is loaded to gpu


In [19]:
# model = retinanet_mobilenet_fpn(pretrained=False, min_size=1024, max_size=1024, num_classes=2)
# torch.save(model.state_dict(), 'ret1.pt')

# model = retinanet_resnet18_fpn(pretrained=False, min_size=1024, max_size=1024, num_classes=2)
# torch.save(model.state_dict(), 'ret2.pt')

# model = retinanet_resnet50_fpn(pretrained=False, min_size=1024, max_size=1024, num_classes=2)
# torch.save(model.state_dict(), 'ret3.pt')


In [20]:
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params, lr=args.lr, weight_decay=args.weight_decay)
# optimizer = torch.optim.SGD(
#        params, lr=args.lr, momentum=args.momentum, weight_decay=args.weight_decay)
lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[20, 40, 60, 80, 100], 
                                                    gamma=0.5)

In [21]:
# evaluate(model, test_loader, device=device)    

In [22]:
from train_lbp import train_one_epoch

start_time = time.time()
for epoch in range(args.epochs):
    train_one_epoch(model, optimizer, train_loader, device, epoch, args.print_freq)
    lr_scheduler.step()
    
    if epoch > 60 and epoch % 5 == 0 :
        if args.output_dir:
            checkpoint = {
                'model': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'lr_scheduler': lr_scheduler.state_dict(),
                'args': args,
                'epoch': epoch
            }
            utils.save_on_master(
                checkpoint,
                os.path.join(args.output_dir, 'model_{}.pth'.format(epoch)))
            utils.save_on_master(
                checkpoint,
                os.path.join(args.output_dir, 'checkpoint.pth'))

    if epoch > 40 and epoch % 5 == 0 :
        # evaluate after every epoch
        evaluate(model, test_loader, device=device)    
print('total time is {}'.format(time.time() - start_time))    

Epoch: [0]  [  0/353]  eta: 0:31:07  lr: 0.002000  loss: 3.8123 (3.8123)  classification: 1.1369 (1.1369)  bbox_regression: 2.6754 (2.6754)  time: 5.2903  data: 4.9132  max mem: 1876
Epoch: [0]  [352/353]  eta: 0:00:00  lr: 0.002000  loss: 2.3755 (239092.4548)  classification: 0.8934 (239090.2487)  bbox_regression: 1.4975 (2.2123)  time: 0.2883  data: 0.1270  max mem: 1964
Epoch: [0] Total time: 0:02:07 (0.3612 s / it)
Epoch: [1]  [  0/353]  eta: 0:28:12  lr: 0.002000  loss: 2.1778 (2.1778)  classification: 0.8393 (0.8393)  bbox_regression: 1.3384 (1.3384)  time: 4.7935  data: 4.5900  max mem: 1964
Epoch: [1]  [352/353]  eta: 0:00:00  lr: 0.002000  loss: 3.5104 (583.6978)  classification: 0.8425 (0.8343)  bbox_regression: 2.6661 (582.8634)  time: 0.2779  data: 0.1182  max mem: 1964
Epoch: [1] Total time: 0:02:09 (0.3665 s / it)
Epoch: [2]  [  0/353]  eta: 0:30:30  lr: 0.002000  loss: 3.1999 (3.1999)  classification: 0.8493 (0.8493)  bbox_regression: 2.3506 (2.3506)  time: 5.1861  data:

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/beomgon/anaconda3/envs/pytorch_retina/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-22-28b577864034>", line 26, in <module>
    evaluate(model, test_loader, device=device)
  File "/home/beomgon/anaconda3/envs/pytorch_retina/lib/python3.7/site-packages/torch/autograd/grad_mode.py", line 28, in decorate_context
    return func(*args, **kwargs)
  File "../engine.py", line 81, in evaluate
    coco = get_coco_api_from_dataset(data_loader.dataset)
  File "../coco_utils.py", line 206, in get_coco_api_from_dataset
    return convert_to_coco_api(dataset)
  File "../coco_utils.py", line 155, in convert_to_coco_api
    img, targets = ds[img_idx]
  File "../dataset.py", line 83, in __getitem__
    image = cv2.imread(self.default_path + path)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceb

TypeError: object of type 'NoneType' has no len()

In [ ]:
model.eval()
images, targets = next(iter(test_loader))
images = list(img.to(device) for img in images)
with torch.no_grad():
    outputs = model(images)
    outputs = [{k: v.to(device) for k, v in t.items()} for t in outputs]
    
images = [image.to('cpu') for image in images]
outputs = [{k: v.to('cpu') for k, v in t.items()} for t in outputs]    


In [ ]:
outputs

In [ ]:
IMAGE_NUM = 0
targets[IMAGE_NUM]['image_id'].item()

In [ ]:
from visualize import visualize
IMAGE_NUM = 2
image = images[IMAGE_NUM].numpy()
image = np.transpose(image, (1, 2, 0))
img_id = targets[IMAGE_NUM]['image_id'].item()
img_path = df[df['ID']==img_id].file_name.values[0]
print(img_path)
ground_boxes = targets[IMAGE_NUM]['boxes']
out_boxes = outputs[IMAGE_NUM]['boxes']
out_scores = outputs[IMAGE_NUM]['scores']
print(out_scores)

pred_boxes = []
for b, s in zip(out_boxes, out_scores) :
    if s > 0.5 :
        pred_boxes.append(b.numpy())

        
import cv2
abs_path = '/home/NAS/nas4/project_scl/'
image = cv2.imread(abs_path + img_path)
visualize(image, ground_boxes, pred_boxes)        